<a href="https://colab.research.google.com/github/abSETEd/check_phone/blob/main/checkphone_verwww.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Versão 1 finalizada

In [ ]:
# Instala as dependências necessárias
!pip install beautifulsoup4
!pip install openpyxl
!pip install requests
!pip install pandas
!pip install pyexcel-ods3


import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import openpyxl  # Para XLSX
from google.colab import files  # Para o upload no Colab
import io  # Para processar o arquivo enviado
import re  # Para expressões regulares
import os  # Pra manipular nomes de arquivo
import pyexcel_ods3

# Função para carregar a planilha em diferentes formatos
def carregar_planilha(arquivo_carregado, nome_arquivo):
    try:
        if nome_arquivo.endswith('.csv'):
            df = pd.read_csv(io.BytesIO(arquivo_carregado))
        elif nome_arquivo.endswith('.xlsx'):
            df = pd.read_excel(io.BytesIO(arquivo_carregado), engine='openpyxl')
        elif nome_arquivo.endswith('.ods'):
            ods_data = pyexcel_ods3.get_data(io.BytesIO(arquivo_carregado))
            df = pd.DataFrame(ods_data[list(ods_data.keys())[0]][1:], columns=ods_data[list(ods_data.keys())[0]][0])
        else:
            raise ValueError("Formato de arquivo não suportado. Use CSV, XLSX ou ODS.")

        colunas_esperadas = ["Site", "qtd CNPJs", "qtd UF MVX", "CNPJ", "CNPJRaiz", "RazaoSocial", "Logradouro",
                            "Numero2", "Complemento", "Bairro", "Cidade", "Uf", "Cep", "CapitalSocial",
                            "Cnae", "CodigoPredio", "Predio", "NaturezaJuridica", "DescNaturezaJuridica",
                            "Mailing", "SDR"]

        colunas_faltantes = [col for col in colunas_esperadas if col not in df.columns]
        if colunas_faltantes:
          raise ValueError(f"As colunas {colunas_faltantes} não foram encontradas na planilha")

        if 'Site' not in df.columns:
          raise ValueError("A planilha precisa ter uma coluna chamada 'Site'.")

        # Formata a coluna 'Site' adicionando 'https://' se necessário
        df['Site'] = df['Site'].apply(lambda x: f"https://www.{x}" if not str(x).startswith(('http://', 'https://')) else x)

        return df
    except Exception as e:
        print(f"Erro ao carregar a planilha: {e}")
        return None

# Função para consultar um site e extrair telefones
def consultar_site(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        resposta = requests.get(url, headers=headers, timeout=10)
        resposta.raise_for_status()

        soup = BeautifulSoup(resposta.text, 'html.parser')
        texto = soup.get_text()

        # Padrão flexível pra telefones no Brasil
        padrao_telefone = r'(?:(?:\+?55\s?)?(?:\(?\d{2}\)?)?\s?\d{4,5}[-.\s]?\d{4})'
        telefones = re.findall(padrao_telefone, texto)
        telefones_unicos = list(set(telefones))

        if telefones_unicos:
            return ', '.join(telefones_unicos)  # Junta os telefones com vírgula
        else:
            return "Nenhum telefone encontrado"
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar {url}: {e}")
        return "Erro na consulta"

# Função principal
def main():
    print("Por favor, faça o upload da sua planilha (CSV, XLSX ou ODS):")

    # Interface de upload
    uploaded = files.upload()  # Abre o seletor de arquivos

    if not uploaded:
        print("Nenhum arquivo foi enviado. Encerrando.")
        return

    # Pega o nome e conteúdo do arquivo enviado
    nome_arquivo = list(uploaded.keys())[0]
    arquivo_carregado = uploaded[nome_arquivo]

    # Carrega a planilha e formata URLs
    df_urls = carregar_planilha(arquivo_carregado, nome_arquivo)
    if df_urls is None:
        return

    print(f"Carreguei {len(df_urls)} linhas da planilha: {nome_arquivo}")

    telefones = []
    for index, row in df_urls.iterrows():
        url = row['Site']
        print(f"Consultando: {url}")

        telefone = consultar_site(url)
        telefones.append(telefone)
        print(f"Telefones para {url}: {telefone}")

        time.sleep(2)

    df_urls['Telefone'] = telefones

    colunas_antes = df_urls.columns[:df_urls.columns.get_loc('RazaoSocial') + 1].tolist()
    colunas_depois = df_urls.columns[df_urls.columns.get_loc('Logradouro'):].tolist()
    novas_colunas = colunas_antes + ['Telefone'] + colunas_depois

    df_urls = df_urls[novas_colunas]

    while True:
      formato = input("Escolha o formto de saída (csv, xlsx, ods): ").lower().strip()
      if formato in ['csv', 'xlsx', 'ods']:
        break
      print("Formato invalido! escolha entre 'csv', 'xlsx' ou 'ods'. " )

    nome_base = os.path.splitext(nome_arquivo)[0]
    if formato == 'csv':
        arquivo_saida = f"{nome_base}_com_telefones.csv"
        df_urls.to_csv(arquivo_saida, index=False)
        print(f'Arquivo salvo como {arquivo_saida} com {len(df_urls)} entradas.')
        files.download(arquivo_saida)
        print(f" Baixe a planilha aqui {arquivo_saida}")
    elif formato == 'xlsx':
        arquivo_saida = f"{nome_base}_com_telefone.xlsx"
        df_urls.to_excel(arquivo_saida, index=False, engine='openpyxl')
        print(f'Arquivo salvo como {arquivo_saida} com {len(df_urls)} entradas.')
        files.download(arquivo_saida)
        print(f" Baixe a planilha aqui: {arquivo_saida}")
    elif formato == 'ods':
        arquivo_saida = f"{nome_base}_com_telefones.ods"
        df_urls = df_urls.fillna('')
        pyexcel_ods3.save_data(arquivo_saida, {'Sheet1': [df_urls.columns.tolist()] + df_urls.values.tolist()})
        print(f'Arquio salvo como {arquivo_saida} com {len(df_urls)} entradas.')
        files.download(arquivo_saida)
        print(f" Baixe a planilha aqui: {arquivo_saida}")

# Executa o programa
if __name__ == "__main__":
    main()

> Documentação

O codigo, em medi, consegue pesquisar 1/3 dos sites.

Ao verificar e validar alguns pontos pesquisando as urls que o codigo não conseguiu verificar e trazer o telefone percebi que:


1.   sites que o telefone fica em outra pagina o bs4 nao consegue buscar, tenho que encontrar outras formas de pesquisa ou utilisar selenium
2.   algumas paginas tem proteção contra web scraping que nao permite a execução do codigo.
3.  Percebi também que numeros com 0800 não são trazidos pelo codigo.
